In [1]:
import numpy as np
import pysindy as ps
import timeit
from scipy.io import loadmat
from sklearn.metrics import mean_squared_error

In [2]:
dt=loadmat('lorenz_cos_0.000003.mat')['delta_t'][0][0]
x=loadmat('lorenz_cos_0.000003.mat')['tilde_u'][:,:,np.newaxis].transpose(1,0,2)
t=dt*np.arange(x.shape[1])
shape=np.concatenate([x.shape[:-1],[4]])
u=np.zeros(shape)
u[:,:,0]=np.cos(t)
u[:,:,1]=(np.roll(x,-1,axis=0))[:,:,0]#u_{j+1}
u[:,:,2]=(np.roll(x,2,axis=0))[:,:,0]
u[:,:,3]=(np.roll(x,1,axis=0))[:,:,0]
true_coefs=np.array([8,-1,-1,1,-1,1])
true_c = np.zeros([56,])
true_c[0] = true_coefs[0]
true_c[1] = true_coefs[1]
true_c[17] = true_coefs[2]
true_c[19] = true_coefs[3]
true_c[42] = true_coefs[4]
true_c[44] = true_coefs[5]

In [3]:
start=timeit.default_timer()
lib=ps.PolynomialLibrary(degree=3)

In [4]:
opt=ps.STLSQ(alpha=1e-6,threshold=5e-1)
model = ps.SINDy(feature_library=lib,optimizer=opt)
model.fit(x, t=dt, u=u)
stop=timeit.default_timer()
model.print(precision=5)
print("Runtime: ", stop-start)
print("Relative coefficient errors: ",((model.optimizer.coef_[0][[0,1,17,19,42,44]]-true_coefs)**2/true_coefs**2)**0.5)
print("Total error: ",np.linalg.norm((model.optimizer.coef_[0]-true_c), ord=None, axis=None, keepdims=False)/np.linalg.norm(true_c))

(x0)' = 8.00001 1 + -0.99998 x0 + -1.00077 u1 u3 + 1.00169 u2 u3 + -0.99921 u0 u1 u3 + 0.99824 u0 u2 u3
Runtime:  0.1084495999966748
Relative coefficient errors:  [1.17461019e-06 2.48037162e-05 7.67937097e-04 1.68917084e-03
 7.90067800e-04 1.75725134e-03]
Total error:  0.0003220376352438177
